# Assistente legale virtuale


## Importazione librerie

In [1]:
from flask import Flask, render_template, request, jsonify
from flask_ngrok import run_with_ngrok
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext, load_index_from_storage
from llama_index.storage.storage_context import StorageContext
from llama_index.text_splitter import SentenceSplitter
from llama_index.llms import OpenAI
from llama_index.tools import QueryEngineTool, ToolMetadata
from llama_index.agent import ReActAgent
from utils import InitializeOpenAI
import nest_asyncio
import os

## Applicazione
Per realizzare il nostro programma attraverso cui interagire col chatbot, abbiamo utilizzato **Flask**.\
Flask è un framework web open-source per Python che consente di creare applicazioni web in modo semplice ed efficiente.\
Il concetto generale attorno a cui funziona Flask è quello di **routing**, che consiste nell'associare determinate azioni a specifici URL. Si definiscono quindi delle **route** che indicano quale funzione o metodo deve essere eseguito quando viene raggiunto un determinato URL.

Innanzitutto abbiamo definito il nome dell'applicazione: *app*\
La funzione *run_with_ngrok*, messa a disposizione da Flask, permette di eseguire l'applicazione in un notebook.

In [2]:
app = Flask(__name__)
run_with_ngrok(app)


Il modulo **nest_asyncio** è progettato per risolvere problemi di compatibilità tra l'utilizzo di asyncio (Async I/O) e l'esecuzione di codice in ambienti che non sono specificamente progettati per supportare asyncio

In [3]:
nest_asyncio.apply()

### Funzioni
In questa sezione abbiamo definito le funzioni che vengono poi chiamate durante l'esecuzione dell'applicazione. Le funzioni create sono 3:
- *SettingVectorIndices(topics_list, service_context)*
- *LoadVectorIndex(topics_list, service_context)*
- *IndividualQueryEngineTools(topic_list, index_list)*

*SettingVectorIndices(topics_list, service_context)* prende in input *topics_list*, che è una lista composta dalle stringhe corrispondenti agli argomenti trattati dai documenti che vogliamo indicizzare, e *service_context*. Quest'ultimo è un oggetto creato dalla funzione *ServiceContext()* e serve a dare al nostro modello la conoscenza di base della rete LLM da noi scelta.\
*ServiceContext()* è una classe messa a disposizione da LLamaindex per ottenere la base knowledge di una LLM e personalizzare il processo di indicizzazione dei documenti. Permette infatti di decidere quale LLM utilizzare, di scrivere un prompt che istruisca la rete per l'operazione di parsing, di specificare quale modello di embedding utilizzare e altro. L'output di SettingVectorIndices è un dizionario che racchiude i vector index dei due documenti.

In [4]:
def SettingVectorIndices(topics_list, service_context):
    doc_set = {}
    all_docs = []
    
    for topic in topics_list:
        reader = SimpleDirectoryReader(input_files=[f"C:/Users/Raffa/Desktop/Text_mining/memoria/{topic}.txt"])
        topic_docs = reader.load_data()
        # insert topic metadata into each topic
        for d in topic_docs:
            d.metadata = {"topic": topic}
        doc_set[topic] = topic_docs
        all_docs.extend(topic_docs)

    index_set = {}
    splitter = SentenceSplitter(separator="CIVIL CODE")

    for topic in topics_list:
        nodes = splitter.get_nodes_from_documents(doc_set[topic])
        cur_index = VectorStoreIndex.from_documents(
            doc_set[topic],
            service_context=service_context,
            nodes = nodes,
        )
        index_set[topic] = cur_index
        cur_index.storage_context.persist(persist_dir=f"./storage/{topic}")


    return index_set

*LoadVectorIndex(topics_list, service_context)* è una funzione creata per quando si vogliono caricare gli indici già presenti nello storage e che permette quindi di evitare di utilizzare ad ogni sessione un LLM per ricostruire gli indici per degli specifici documenti.

In [5]:
def LoadVectorIndex(topics_list, service_context):
    index_set = {}
    for topic in topics_list:
        storage_context = StorageContext.from_defaults(
            persist_dir=f"./storage/{topic}"
        )
        cur_index = load_index_from_storage(
            storage_context = storage_context, service_context = service_context
        )
        index_set[topic] = cur_index

    return index_set

*IndividualQueryEngineTools(topic_list, index_list)* prende in input, oltre a *topic_list*, *index_list* che non è altro che il return di SettingVectorIndices (o LoadVectorIndex). Questa funzione crea due tool messi a disposizione dell'agent. Un tool serve a ricercare testo pertinente a queries sulla spartizione dei beni tra divorzianti, analogamente l'altro tool per l'eredità.

In [6]:
def IndividualQueryEngineTools(topic_list, index_list):

    devorce = {'What does the civil code say about the obligations of spouses contracted before marriage?': 'As said in Article 211, the community property are liable for obligations contracted by one of the spouses before the marriage limited to the value of the property owned by that spouse before the marriage which, by agreement entered into, became part of the community of property.',
           'If I divorce my spouse, when is community property to be considered terminated?': 'As said in the Article 191, in the case of legal separation, the community of property between the spouses is dissolved at the time when the president of the court authorises the spouses to live separately, or on the date of the signing of the minutes of the consensual separation of the spouses before the president, provided that they have been approved.',
           'If my spouse squanders family property, can I cancel community property?': 'Yes, according to what is stated in Article 193 of the Civil Code. Provided that the conduct of the spouse engaged in the administration of property endangers the interests of the other or the community or family.'}
    inheritance = {"If I am entitled to a person's inheritance but die before accepting it, will my children be entitled to said inheritance?": "Yes, because as mentioned in Article 479 of the Civil Code, if the person called to the inheritance dies without accepting it, the right to accept it is passed on to the heirs."}
    individual_query_engine_tools = []

    devorce_examples = ""
    inheritance_examples = ""

    for query, answer in devorce.items():
        devorce_examples += f"{query}\n{answer}\n\n"

    for query, answer in inheritance.items():
        inheritance_examples += f"{query}\n{answer}\n\n"

    examples_list = [devorce_examples, inheritance_examples]

    for i,topic in enumerate(topic_list):
        query_engine = index_list[topic].as_query_engine()

        metadata = ToolMetadata(
                name=f"vector_index_{topic}",
                description=f"If you can, when answering questions regarding laws of the Civil Code also mention the number of the article you are referring to. Below are some examples.\n\n{examples_list[i]}",
            )

        tool_instance = QueryEngineTool(query_engine=query_engine, metadata=metadata)
        individual_query_engine_tools.append(tool_instance)


    return individual_query_engine_tools

### Metodi dell'applicazione *app*
Innanzitutto abbiamo definito una route per l'URL principale **'/'**. Quando l'applicazione Flask riceve una richiesta per questa route, eseguirà la funzione *index()* e restituirà il risultato di *render_template('index.html')*, cioè la pagina principale dell'applicazione.\
Dopo abbiamo definito una route per l'URL **/chat** che accetta solo richieste POST *(methods=['POST'])*.
Quando l'applicazione Flask riceve una richiesta POST a questa route, eseguirà la funzione *chat()*.
All'interno di chat(), si ottiene l'input dell'utente dalla richiesta POST: 

user_input = request.form['user_input']

Quindi, utilizzando l'oggetto agent, viene chiamato il metodo chat() con l'input dell'utente per ottenere una risposta:

response = agent.chat(user_input)

La risposta ottenuta viene quindi estratta dal campo response dell'oggetto response: 

response_text = response.response.

Infine, viene restituita una risposta JSON contenente il testo della risposta: 

return jsonify({'response': response_text})

In [7]:
@app.route('/')
def index():
    return render_template('index.html')

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.form['user_input']
    response = agent.chat(user_input)

    # Estrai il testo dalla risposta
    response_text = response.response

    return jsonify({'response': response_text})



## Main del codice
Il codice mostrato sotto è il main che a sua volta chiama le funzioni viste finora a parte *InitializeOpenAI()* che è una funzione presente nel file *utils.py*. Ad ogni modo, InitializeOpenAI() definisce semplicemente la API key di OpenAI (di cui usufruiamo i servizi utilizzando il suo modello "gpt-3.5-turbo-0613" in ServiceContext e come response synthetizer) come variabile d'ambiente.\
Quindi, come si può vedere abbiamo utilizzato "gpt-3.5-turbo-0613" come modello per effettuare l'embedding e la sintesi delle risposte.\
L'if statement gestisce le due eventualità: caricare indexes presenti nello storage o rifarli daccapo.

In [8]:
if __name__ == "__main__":
    InitializeOpenAI()
    llm = OpenAI(model="gpt-3.5-turbo-0613")
    service_context = ServiceContext.from_defaults(llm=llm)

    storage_path = "storage"
    topics = ['DIVISION_OF_ASSETS_AFTER_DIVORCE', 'INHERITANCE']

    if len(os.listdir(storage_path)) > 0:
        index_set = LoadVectorIndex(topics, service_context)
    else:
        index_set = SettingVectorIndices(topics)

    individual_query_engine_tools = IndividualQueryEngineTools(topics, index_set)

    tools = individual_query_engine_tools
    agent = ReActAgent.from_tools(tools, llm=llm, verbose=True)

    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [21/Dec/2023 12:49:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Dec/2023 12:49:22] "GET /favicon.ico HTTP/1.1" 404 -
Exception in thread Thread-4:
Traceback (most recent call last):
  File "c:\Jupyter\envs\Text_Mining\Lib\threading.py", line 1038, in _bootstrap_inner
    self.run()
  File "c:\Jupyter\envs\Text_Mining\Lib\threading.py", line 1394, in run
    self.function(*self.args, **self.kwargs)
  File "c:\Jupyter\envs\Text_Mining\Lib\site-packages\flask_ngrok.py", line 70, in start_ngrok
    ngrok_address = _run_ngrok()
                    ^^^^^^^^^^^^
  File "c:\Jupyter\envs\Text_Mining\Lib\site-packages\flask_ngrok.py", line 38, in _run_ngrok
    tunnel_url = j['tunnels'][0]['public_url']  # Do the parsing of the get
                 ~~~~~~~~~~~~^^^
IndexError: list index out of range
127.0.0.1 - - [21/Dec/2023 12:51:05] "POST /chat HTTP/1.1" 200 -


: 